08Junho20 - dbSigaContemporaneo

In [ ]:
-- Trigger criada pelo Activesoft Service Pack em 09/04/2019
-- durante atualização para versão 1.064.907.
CREATE TRIGGER dbo.TgAudit_TbDiarioI ON dbo.TbDiario FOR INSERT AS 
BEGIN
	SET NOCOUNT ON

	IF ( SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS c WHERE DATA_TYPE NOT IN ( 'text', 'image' ) AND TABLE_NAME = 'TbDiario' AND LEFT(COLUMN_NAME,1) <> '_' ) <> 11 
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiario inválida)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 


	IF ISNULL( (	SELECT	SUM( CASE WHEN c.COLUMN_NAME IN ( '','IdDiario' ) THEN 1 ELSE 100 END )
			FROM	INFORMATION_SCHEMA.TABLE_CONSTRAINTS pk, 
				INFORMATION_SCHEMA.KEY_COLUMN_USAGE c 
			WHERE	CONSTRAINT_TYPE = 'PRIMARY KEY'
				AND c.TABLE_NAME = pk.TABLE_NAME
				AND c.CONSTRAINT_NAME = pk.CONSTRAINT_NAME
				AND pk.TABLE_NAME = 'TbDiario' ), 0 ) <> 1
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiario inválida -- definição da chave primária)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 


	DECLARE @IdUsuario INTEGER
	IF APP_NAME() = 'SIGAWEB'
		SELECT	@IdUsuario = IdUsuario
		FROM	dbo.FnCriarTabelaTemporaria(HOST_NAME())		
	ELSE IF OBJECT_ID( 'tempdb..#TempTable') IS NULL 
	  BEGIN 
		--RAISERROR ('Dados não podem ser alterados (autenticação)', 16, 1) 
		--ROLLBACK TRANSACTION 
		--RETURN 
		SET @IdUsuario = NULL
	  END
	ELSE
	  SELECT @IdUsuario = IdUsuario FROM #TempTable



	DECLARE	@Descricao VARCHAR(8000), 
		@Value VARCHAR(500) 

	DECLARE xAudit CURSOR FOR 
		SELECT  IdDiario, IdTurma, IdDisciplina, Descricao, IdFaseNota, TipoConfirmacao, DataLimiteDigitacao, QtdeMaximaAulas, QtdeMinimaAulas, StRegistroAulaPeriodoFaseNota, DataBloqueioDigitacaoAula 
		FROM	inserted

	OPEN xAudit
  
	DECLARE @Audit_IdDiario int, @Audit_IdTurma int, @Audit_IdDisciplina int, @Audit_Descricao varchar(50), @Audit_IdFaseNota int, @Audit_TipoConfirmacao char(1), @Audit_DataLimiteDigitacao datetime, @Audit_QtdeMaximaAulas int, @Audit_QtdeMinimaAulas int, @Audit_StRegistroAulaPeriodoFaseNota bit, @Audit_DataBloqueioDigitacaoAula datetime
  
	FETCH NEXT FROM xAudit INTO @Audit_IdDiario, @Audit_IdTurma, @Audit_IdDisciplina, @Audit_Descricao, @Audit_IdFaseNota, @Audit_TipoConfirmacao, @Audit_DataLimiteDigitacao, @Audit_QtdeMaximaAulas, @Audit_QtdeMinimaAulas, @Audit_StRegistroAulaPeriodoFaseNota, @Audit_DataBloqueioDigitacaoAula
	WHILE @@FETCH_STATUS = 0
	  BEGIN
		SET @Descricao = ''


		SET @Value = ISNULL( ( SELECT NomeTurmaCompleto FROM vwTurma WHERE IdTurma = @Audit_IdTurma ) , '<nulo>' )
		SET @Descricao = @Descricao + 'Turma: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( SELECT NomeDisciplina FROM TbDisciplina WHERE IdDisciplina = @Audit_IdDisciplina ), '<nulo>' )
		SET @Descricao = @Descricao + 'Disciplina: ' + @Value + CHAR(13)

		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_Descricao ), '<nulo>' )
		SET @Descricao = @Descricao + 'Descrição: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( SELECT NomeFase FROM TbFaseNota WHERE IdFaseNota = @Audit_IdFaseNota ) , '<nulo>' )
		SET @Descricao = @Descricao + 'Fase de nota: ' + @Value + CHAR(13)

		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_TipoConfirmacao ), '<nulo>' )
		SET @Descricao = @Descricao + 'Situação: ' + @Value + CHAR(13)

		SET @Value = ISNULL( REPLACE( CONVERT( VARCHAR(20), @Audit_DataLimiteDigitacao, 103 ) + ' ' + CONVERT( VARCHAR, @Audit_DataLimiteDigitacao, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @Descricao = @Descricao + 'Data limite para digitação: ' + @Value + CHAR(13)

		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_QtdeMaximaAulas ), '<nulo>' )
		SET @Descricao = @Descricao + 'Qtde máxima de aulas: ' + @Value + CHAR(13)

		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_QtdeMinimaAulas ), '<nulo>' )
		SET @Descricao = @Descricao + 'Qtde mínima de aulas: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( CASE WHEN @Audit_StRegistroAulaPeriodoFaseNota=0 THEN 'Não' ELSE 'Sim' END ), '<nulo>' )
		SET @Descricao = @Descricao + 'StRegistroAulaPeriodoFaseNota: ' + @Value + CHAR(13)

		SET @Value = ISNULL( REPLACE( CONVERT( VARCHAR(20), @Audit_DataBloqueioDigitacaoAula, 103 ) + ' ' + CONVERT( VARCHAR, @Audit_DataBloqueioDigitacaoAula, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @Descricao = @Descricao + 'Data bloqueio para digitação das aulas: ' + @Value + CHAR(13)

		INSERT TbAuditoria ( Tabela, IdUsuario, ValorChave1, ValorChave2, ValorChave3, ValorChave4, ValorChave5, Operacao, DescricaoOperacao )
			SELECT 'TbDiario', @IdUsuario, @Audit_IdDiario, @Audit_IdTurma, NULL, NULL, NULL, 'I', @Descricao

		FETCH NEXT FROM xAudit INTO @Audit_IdDiario, @Audit_IdTurma, @Audit_IdDisciplina, @Audit_Descricao, @Audit_IdFaseNota, @Audit_TipoConfirmacao, @Audit_DataLimiteDigitacao, @Audit_QtdeMaximaAulas, @Audit_QtdeMinimaAulas, @Audit_StRegistroAulaPeriodoFaseNota, @Audit_DataBloqueioDigitacaoAula
	  END

	CLOSE xAudit
	DEALLOCATE xAudit
END


In [ ]:
-- Trigger criada pelo Activesoft Service Pack em 09/04/2019
-- durante atualização para versão 1.064.907.
CREATE TRIGGER dbo.TgAudit_TbDiarioU ON dbo.TbDiario FOR UPDATE AS 
BEGIN
	SET NOCOUNT ON

	IF ( SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS c WHERE DATA_TYPE NOT IN ( 'text', 'image' ) AND TABLE_NAME = 'TbDiario' AND LEFT(COLUMN_NAME,1) <> '_' ) <> 11 
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiario inválida -- quantidade de campos)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 


	IF ISNULL( (	SELECT	SUM( CASE WHEN c.COLUMN_NAME IN ( '','IdDiario' ) THEN 1 ELSE 100 END )
			FROM	INFORMATION_SCHEMA.TABLE_CONSTRAINTS pk, 
				INFORMATION_SCHEMA.KEY_COLUMN_USAGE c 
			WHERE	CONSTRAINT_TYPE = 'PRIMARY KEY'
				AND c.TABLE_NAME = pk.TABLE_NAME
				AND c.CONSTRAINT_NAME = pk.CONSTRAINT_NAME
				AND pk.TABLE_NAME = 'TbDiario' ), 0 ) <> 1
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiario inválida -- definição da chave primária)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 

	  
	DECLARE @IdUsuario INTEGER
	IF APP_NAME() = 'SIGAWEB'
		SELECT	@IdUsuario = IdUsuario
		FROM	dbo.FnCriarTabelaTemporaria(HOST_NAME())		
	ELSE IF OBJECT_ID( 'tempdb..#TempTable') IS NULL 
	  BEGIN 
		--RAISERROR ('Dados não podem ser alterados (autenticação)', 16, 1) 
		--ROLLBACK TRANSACTION 
		--RETURN 
		SET @IdUsuario = NULL
	  END
	ELSE
	  SELECT @IdUsuario = IdUsuario FROM #TempTable


	DECLARE	@Descricao VARCHAR(8000), 
		@StHouveCampoAlterado BIT,
		@OldValue VARCHAR(500), 
		@NewValue VARCHAR(500) 

	DECLARE xAudit CURSOR FOR 
		SELECT  d.IdDiario, i.IdDiario, d.IdTurma, i.IdTurma, d.IdDisciplina, i.IdDisciplina, d.Descricao, i.Descricao, d.IdFaseNota, i.IdFaseNota, d.TipoConfirmacao, i.TipoConfirmacao, d.DataLimiteDigitacao, i.DataLimiteDigitacao, d.QtdeMaximaAulas, i.QtdeMaxi
maAulas, d.QtdeMinimaAulas, i.QtdeMinimaAulas, d.StRegistroAulaPeriodoFaseNota, i.StRegistroAulaPeriodoFaseNota, d.DataBloqueioDigitacaoAula, i.DataBloqueioDigitacaoAula 
		FROM	deleted d 
		FULL	OUTER JOIN inserted i ON d.IdDiario  =  i.IdDiario

	OPEN xAudit 
  
	DECLARE @OLD_IdDiario int, @NEW_IdDiario int, @OLD_IdTurma int, @NEW_IdTurma int, @OLD_IdDisciplina int, @NEW_IdDisciplina int, @OLD_Descricao varchar(50), @NEW_Descricao varchar(50), @OLD_IdFaseNota int, @NEW_IdFaseNota int, @OLD_TipoConfirmacao char(1)
, @NEW_TipoConfirmacao char(1), @OLD_DataLimiteDigitacao datetime, @NEW_DataLimiteDigitacao datetime, @OLD_QtdeMaximaAulas int, @NEW_QtdeMaximaAulas int, @OLD_QtdeMinimaAulas int, @NEW_QtdeMinimaAulas int, @OLD_StRegistroAulaPeriodoFaseNota bit, @NEW_StRegistroAulaPeriodoFaseNota bit, @OLD_DataBloqueioDigitacaoAula datetime, @NEW_DataBloqueioDigitacaoAula datetime
  
	FETCH NEXT FROM xAudit INTO @OLD_IdDiario, @NEW_IdDiario, @OLD_IdTurma, @NEW_IdTurma, @OLD_IdDisciplina, @NEW_IdDisciplina, @OLD_Descricao, @NEW_Descricao, @OLD_IdFaseNota, @NEW_IdFaseNota, @OLD_TipoConfirmacao, @NEW_TipoConfirmacao, @OLD_DataLimiteDigitacao, @NEW_DataLimiteDigitacao, @OLD_QtdeMaximaAulas, @NEW_QtdeMaximaAulas, @OLD_QtdeMinimaAulas, @NEW_QtdeMinimaAulas, @OLD_StRegistroAulaPeriodoFaseNota, @NEW_StRegistroAulaPeriodoFaseNota, @OLD_DataBloqueioDigitacaoAula, @NEW_DataBloqueioDigitacaoAula

	WHILE @@FETCH_STATUS = 0
	  BEGIN
		SET @Descricao = ''
		SET @StHouveCampoAlterado = 0


		SET @OldValue = ISNULL( ( SELECT NomeTurmaCompleto FROM vwTurma WHERE IdTurma = @OLD_IdTurma ) , '<nulo>' )
		SET @NewValue = ISNULL( ( SELECT NomeTurmaCompleto FROM vwTurma WHERE IdTurma = @NEW_IdTurma ) , '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Turma: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		ELSE 	SET @Descricao = @Descricao + 'Turma: ' + @OldValue + CHAR(13)

		SET @OldValue = ISNULL( ( SELECT NomeDisciplina FROM TbDisciplina WHERE IdDisciplina = @OLD_IdDisciplina ), '<nulo>' )
		SET @NewValue = ISNULL( ( SELECT NomeDisciplina FROM TbDisciplina WHERE IdDisciplina = @NEW_IdDisciplina ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Disciplina: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		ELSE 	SET @Descricao = @Descricao + 'Disciplina: ' + @OldValue + CHAR(13)

		SET @OldValue = ISNULL( CONVERT( VARCHAR(500), @OLD_Descricao ), '<nulo>' )
		SET @NewValue = ISNULL( CONVERT( VARCHAR(500), @NEW_Descricao ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Descrição: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		ELSE 	SET @Descricao = @Descricao + 'Descrição: ' + @OldValue + CHAR(13)

		SET @OldValue = ISNULL( ( SELECT NomeFase FROM TbFaseNota WHERE IdFaseNota = @OLD_IdFaseNota ) , '<nulo>' )
		SET @NewValue = ISNULL( ( SELECT NomeFase FROM TbFaseNota WHERE IdFaseNota = @NEW_IdFaseNota ) , '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Fase de nota: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		ELSE 	SET @Descricao = @Descricao + 'Fase de nota: ' + @OldValue + CHAR(13)

		SET @OldValue = ISNULL( CONVERT( VARCHAR(500), @OLD_TipoConfirmacao ), '<nulo>' )
		SET @NewValue = ISNULL( CONVERT( VARCHAR(500), @NEW_TipoConfirmacao ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Situação: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		ELSE 	SET @Descricao = @Descricao + 'Situação: ' + @OldValue + CHAR(13)

		SET @OldValue = ISNULL( REPLACE( CONVERT( VARCHAR(20), @OLD_DataLimiteDigitacao, 103 ) + ' ' + CONVERT( VARCHAR, @OLD_DataLimiteDigitacao, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @NewValue = ISNULL( REPLACE( CONVERT( VARCHAR(20), @NEW_DataLimiteDigitacao, 103 ) + ' ' + CONVERT( VARCHAR, @NEW_DataLimiteDigitacao, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Data limite para digitação: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		SET @OldValue = ISNULL( CONVERT( VARCHAR(500), @OLD_QtdeMaximaAulas ), '<nulo>' )
		SET @NewValue = ISNULL( CONVERT( VARCHAR(500), @NEW_QtdeMaximaAulas ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Qtde máxima de aulas: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		SET @OldValue = ISNULL( CONVERT( VARCHAR(500), @OLD_QtdeMinimaAulas ), '<nulo>' )
		SET @NewValue = ISNULL( CONVERT( VARCHAR(500), @NEW_QtdeMinimaAulas ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Qtde mínima de aulas: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		SET @OldValue = ISNULL( ( CASE WHEN @OLD_StRegistroAulaPeriodoFaseNota=0 THEN 'Não' ELSE 'Sim' END ), '<nulo>' )
		SET @NewValue = ISNULL( ( CASE WHEN @NEW_StRegistroAulaPeriodoFaseNota=0 THEN 'Não' ELSE 'Sim' END ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'StRegistroAulaPeriodoFaseNota: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		SET @OldValue = ISNULL( REPLACE( CONVERT( VARCHAR(20), @OLD_DataBloqueioDigitacaoAula, 103 ) + ' ' + CONVERT( VARCHAR, @OLD_DataBloqueioDigitacaoAula, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @NewValue = ISNULL( REPLACE( CONVERT( VARCHAR(20), @NEW_DataBloqueioDigitacaoAula, 103 ) + ' ' + CONVERT( VARCHAR, @NEW_DataBloqueioDigitacaoAula, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		IF @OldValue <> @NewValue
		  BEGIN
			SET @Descricao = @Descricao + 'Data bloqueio para digitação das aulas: ' + @OldValue + ' --> ' + @NewValue + CHAR(13)
			SET @StHouveCampoAlterado = 1
		  END
		

		IF ( @StHouveCampoAlterado = 1 ) AND ( @Descricao <> '' )
			INSERT TbAuditoria ( Tabela, IdUsuario, ValorChave1, ValorChave2, ValorChave3, ValorChave4, ValorChave5, Operacao, DescricaoOperacao )
				SELECT 'TbDiario', @IdUsuario, @NEW_IdDiario, @NEW_IdTurma, NULL, NULL, NULL, 'U', @Descricao

		FETCH NEXT FROM xAudit INTO @OLD_IdDiario, @NEW_IdDiario, @OLD_IdTurma, @NEW_IdTurma, @OLD_IdDisciplina, @NEW_IdDisciplina, @OLD_Descricao, @NEW_Descricao, @OLD_IdFaseNota, @NEW_IdFaseNota, @OLD_TipoConfirmacao, @NEW_TipoConfirmacao, @OLD_DataLimiteDigitacao, @NEW_DataLimiteDigitacao, @OLD_QtdeMaximaAulas, @NEW_QtdeMaximaAulas, @OLD_QtdeMinimaAulas, @NEW_QtdeMinimaAulas, @OLD_StRegistroAulaPeriodoFaseNota, @NEW_StRegistroAulaPeriodoFaseNota, @OLD_DataBloqueioDigitacaoAula, @NEW_DataBloqueioDigitacaoAula
	  END


	CLOSE xAudit
	DEALLOCATE xAudit

END


In [ ]:
-- Trigger criada pelo Activesoft Service Pack em 09/04/2019
-- durante atualização para versão 1.064.907.
CREATE TRIGGER dbo.TgAudit_TbDiarioD ON dbo.TbDiario FOR DELETE AS 
BEGIN
	SET NOCOUNT ON

	IF ( SELECT COUNT(*) FROM INFORMATION_SCHEMA.COLUMNS c WHERE DATA_TYPE NOT IN ( 'text', 'image' ) AND TABLE_NAME = 'TbDiario' AND LEFT(COLUMN_NAME,1) <> '_' ) <> 11 
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiario inválida)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 


	IF ISNULL( (	SELECT	SUM( CASE WHEN c.COLUMN_NAME IN ( '','IdDiario' ) THEN 1 ELSE 100 END )
			FROM	INFORMATION_SCHEMA.TABLE_CONSTRAINTS pk, 
				INFORMATION_SCHEMA.KEY_COLUMN_USAGE c 
			WHERE	CONSTRAINT_TYPE = 'PRIMARY KEY'
				AND c.TABLE_NAME = pk.TABLE_NAME
				AND c.CONSTRAINT_NAME = pk.CONSTRAINT_NAME
				AND pk.TABLE_NAME = 'TbDiario' ), 0 ) <> 1
	  BEGIN
		RAISERROR ('Dados não podem ser alterados (trigger de auditoria TgAudit_TbDiario inválida -- definição da chave primária)', 16, 1) 
		ROLLBACK TRANSACTION 
		RETURN 
	  END 	  
	  
	  
	DECLARE @IdUsuario INTEGER
	IF APP_NAME() = 'SIGAWEB'
		SELECT	@IdUsuario = IdUsuario
		FROM	dbo.FnCriarTabelaTemporaria(HOST_NAME())		
	ELSE IF OBJECT_ID( 'tempdb..#TempTable') IS NULL 
	  BEGIN 
		--RAISERROR ('Dados não podem ser alterados (autenticação)', 16, 1) 
		--ROLLBACK TRANSACTION 
		--RETURN 
		SET @IdUsuario = NULL
	  END
	ELSE
	  SELECT @IdUsuario = IdUsuario FROM #TempTable



	DECLARE	@Descricao VARCHAR(8000), 
		@Value VARCHAR(500) 

	DECLARE xAudit CURSOR FOR 
		SELECT  IdDiario, IdTurma, IdDisciplina, Descricao, IdFaseNota, TipoConfirmacao, DataLimiteDigitacao, QtdeMaximaAulas, QtdeMinimaAulas, StRegistroAulaPeriodoFaseNota, DataBloqueioDigitacaoAula 
		FROM	deleted

	OPEN xAudit
  
	DECLARE @Audit_IdDiario int, @Audit_IdTurma int, @Audit_IdDisciplina int, @Audit_Descricao varchar(50), @Audit_IdFaseNota int, @Audit_TipoConfirmacao char(1), @Audit_DataLimiteDigitacao datetime, @Audit_QtdeMaximaAulas int, @Audit_QtdeMinimaAulas int, @Audit_StRegistroAulaPeriodoFaseNota bit, @Audit_DataBloqueioDigitacaoAula datetime
  
	FETCH NEXT FROM xAudit INTO @Audit_IdDiario, @Audit_IdTurma, @Audit_IdDisciplina, @Audit_Descricao, @Audit_IdFaseNota, @Audit_TipoConfirmacao, @Audit_DataLimiteDigitacao, @Audit_QtdeMaximaAulas, @Audit_QtdeMinimaAulas, @Audit_StRegistroAulaPeriodoFaseNota, @Audit_DataBloqueioDigitacaoAula
	WHILE @@FETCH_STATUS = 0
	  BEGIN
		SET @Descricao = ''


		SET @Value = ISNULL( ( SELECT NomeTurmaCompleto FROM vwTurma WHERE IdTurma = @Audit_IdTurma ) , '<nulo>' )
		SET @Descricao = @Descricao + 'Turma: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( SELECT NomeDisciplina FROM TbDisciplina WHERE IdDisciplina = @Audit_IdDisciplina ), '<nulo>' )
		SET @Descricao = @Descricao + 'Disciplina: ' + @Value + CHAR(13)

		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_Descricao ), '<nulo>' )
		SET @Descricao = @Descricao + 'Descrição: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( SELECT NomeFase FROM TbFaseNota WHERE IdFaseNota = @Audit_IdFaseNota ) , '<nulo>' )
		SET @Descricao = @Descricao + 'Fase de nota: ' + @Value + CHAR(13)

		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_TipoConfirmacao ), '<nulo>' )
		SET @Descricao = @Descricao + 'Situação: ' + @Value + CHAR(13)

		SET @Value = ISNULL( REPLACE( CONVERT( VARCHAR(20), @Audit_DataLimiteDigitacao, 103 ) + ' ' + CONVERT( VARCHAR, @Audit_DataLimiteDigitacao, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @Descricao = @Descricao + 'Data limite para digitação: ' + @Value + CHAR(13)

		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_QtdeMaximaAulas ), '<nulo>' )
		SET @Descricao = @Descricao + 'Qtde máxima de aulas: ' + @Value + CHAR(13)

		SET @Value = ISNULL( CONVERT( VARCHAR(500), @Audit_QtdeMinimaAulas ), '<nulo>' )
		SET @Descricao = @Descricao + 'Qtde mínima de aulas: ' + @Value + CHAR(13)

		SET @Value = ISNULL( ( CASE WHEN @Audit_StRegistroAulaPeriodoFaseNota=0 THEN 'Não' ELSE 'Sim' END ), '<nulo>' )
		SET @Descricao = @Descricao + 'StRegistroAulaPeriodoFaseNota: ' + @Value + CHAR(13)

		SET @Value = ISNULL( REPLACE( CONVERT( VARCHAR(20), @Audit_DataBloqueioDigitacaoAula, 103 ) + ' ' + CONVERT( VARCHAR, @Audit_DataBloqueioDigitacaoAula, 114 ), ' 00:00:00:000', '' ), '<nulo>' )
		SET @Descricao = @Descricao + 'Data bloqueio para digitação das aulas: ' + @Value + CHAR(13)

		INSERT TbAuditoria ( Tabela, IdUsuario, ValorChave1, ValorChave2, ValorChave3, ValorChave4, ValorChave5, Operacao, DescricaoOperacao )
			SELECT 'TbDiario', @IdUsuario, @Audit_IdDiario, @Audit_IdTurma, NULL, NULL, NULL, 'D', @Descricao

		FETCH NEXT FROM xAudit INTO @Audit_IdDiario, @Audit_IdTurma, @Audit_IdDisciplina, @Audit_Descricao, @Audit_IdFaseNota, @Audit_TipoConfirmacao, @Audit_DataLimiteDigitacao, @Audit_QtdeMaximaAulas, @Audit_QtdeMinimaAulas, @Audit_StRegistroAulaPeriodoFaseNota, @Audit_DataBloqueioDigitacaoAula
	  END

	CLOSE xAudit
	DEALLOCATE xAudit
END
